In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
print(os.getcwd())
##
import pandas as pd
# These lines in VSCode to avoid reloading the kernel when changes are made to the external kernel
%load_ext autoreload
%autoreload 2

/home/nicolas/codeRepo/OceanPlatformControl


In [65]:
import wandb
api = wandb.Api()

In [83]:
experiment = "experiment.id"
runs = api.runs(path="nhoischen/Master Thesis",
         filters={"$and": [{"config.experiment.id.value": "multi_ag_4_HC_HC"}, {"config.ctrlConfig.high_level_ctrl": "flocking"}]}
)

In [84]:
metrics_dict = {}
for nb_run, run in enumerate(runs):
    try:
        metrics_table = run.logged_artifacts()[0] # my metric table is logged as the first artifact object
        table_dir = metrics_table.download()
        table_path = f"{table_dir}/metrics.table.json"
        #metrics_url = run.summary['metrics']['path']
        metrics = json.load(open(table_path))
        for i, column in enumerate(metrics["columns"]):
            if nb_run == 0:
                metrics_dict[column] = [metrics["data"][0][i]]
            else:
                metrics_dict[column].append(metrics["data"][0][i])
    except BaseException:
        print(f"unable to find metrics for {run.id}")
        continue

df_metrics = pd.DataFrame(metrics_dict)

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downlo

In [85]:
df_metrics

,Isolated_platform_metric,Number_of_collision,Reaching_target,Mean minimum distance to target among all platforms in deg,Mean maximum correction from optimal control degrees,Average Beta Index,Initial maximum degree of the graph,Final maximum degree of the graph,Mission_success
0,0.000000,0,1.00,0.000000,12.564329,1.500000,3,3,1
1,0.047685,0,0.75,0.002309,69.736959,1.328363,2,3,0
2,0.000000,0,1.00,0.000000,20.160268,0.927531,1,3,1
3,0.000000,0,1.00,0.000000,21.468382,1.500000,3,3,1
4,0.000000,0,1.00,0.000000,29.886211,1.317614,3,3,1
...,...,...,...,...,...,...,...,...,...
543,0.000000,0,1.00,0.000000,8.448330,1.486824,3,3,1
544,0.000000,0,1.00,0.000000,5.850589,1.500000,3,3,1
545,0.000000,0,1.00,0.000000,15.461375,1.155687,3,3,1
546,0.000000,0,1.00,0.000000,12.349398,1.500000,3,3,1


In [86]:
df_metrics.to_csv("metrics_4_GOM_flocking_HC_HC.csv")